In [ ]:
import sys
from utils.helpers import *
from dataset import CustomImageFolder
from torchvision import transforms
from functools import partial
import torch
from functions import *
from torchvision.datasets import ImageFolder
from tqdm.auto import tqdm
import csv
from setting import *

set_seeds(2024)
mi = "mi"
device = torch.device("cuda:0")
for model_choice in ["hidden"]:
    for data_choice in ["CelebA"]:
        data_dir = {"COCO":MSCOCO_TEST_PATH,"CelebA":CELEBAHQ_VAL_PATH}
        ckp_dir = f"weights/{mi}weight/{data_choice}/{model_choice}"
        transform_pipe = [
            transforms.Resize((128,128)),
            transforms.ToTensor(),
        ]
        normalize_ = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])
        un_normalize_ = transforms.Normalize([-1,-1,-1],[2,2,2])
        if model_choice == 'hidden':
            transform_pipe.append(transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5]))
        transform = transforms.Compose(transform_pipe)
        if data_choice == "CelebA":
            ds = ImageFolder(data_dir[data_choice],transform=transform)
        else:
            ds = CustomImageFolder(data_dir[data_choice],transform,num=1000)

        # functions
        get_wmimages = partial(get_wmimages,ds=ds,device=device,model_choice=model_choice)
        get_images = partial(get_images,ds=ds,device=device)
        extract_message = partial(extract_message,device=device,model_choice=model_choice)
        csv_file = f"xyattackresults/oriresults/{data_choice}_{model_choice}_{mi}.csv"

        # attack
        message_len = 100
        ckp_list = sorted(os.listdir(ckp_dir))

        if f"{mi}OO" in ckp_list:
            ckp_list.remove(f"{mi}OO")
            ckp_list.insert(0,f"{mi}OO")

        for op in ckp_list:
            if op == "OO":
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state100.pth')
            elif op == 'emperical':
                ckp_path = os.path.join(ckp_dir,op,'epoch_499_state.pth')
            else:
                ckp_path = os.path.join(ckp_dir,op,'epoch_99_state.pth')

            encoder,decoder = load_weights(ckp_path,model_choice,message_len)
            encoder.eval(),decoder.eval()
            targe_message = generate_message(message_len,batch_size=1)
            for image_i in range(1000):
                wm_images,residual_predictions = get_wmimages(text=targe_message,image_i=range(image_i,image_i+1),encoder=encoder)
                residual_predictions_m = residual_predictions.mean(dim=0)
                # similate 
                init_message = "0" * message_len
                temp = init_message
                for bit in tqdm(range(message_len)):
                    temp1 = temp[:bit] + "1" + temp[bit+1:]
                    temp_tensor = generate_message(message_len,temp,batch_size=1)
                    temp1_tensor = generate_message(message_len,temp1,batch_size=1)
                    _,residual_predictions_0 = get_wmimages(text=temp_tensor,image_i=range(image_i,image_i+1),encoder=encoder)
                    _,residual_predictions_1 = get_wmimages(text=temp1_tensor,image_i=range(image_i,image_i+1),encoder=encoder)
                    
                    residual_predictions_0_m = residual_predictions_0.mean(dim=0)
                    residual_predictions_1_m = residual_predictions_1.mean(dim=0)

                    assert residual_predictions_0_m.size() == residual_predictions_1_m.size() == (3,128,128)

                    dis0 = torch.norm(residual_predictions_m - residual_predictions_0_m).item()
                    dis1 = torch.norm(residual_predictions_m - residual_predictions_1_m).item()
                    if dis0 < dis1:
                        temp = temp
                    else:
                        temp = temp1
                
                bit_acc = (message_len -  hamming_distance(temp,msg2str(targe_message))) / message_len
                csv_dict = {"model":op,"bit_acc":bit_acc}
                with open(csv_file, mode='a', newline='') as file:
                    fieldnames = csv_dict.keys()
                    writer = csv.DictWriter(file, fieldnames=fieldnames)
                    if file.tell() == 0:
                        writer.writeheader()
                    writer.writerow(csv_dict)
                print("op:",op)
                print("temp:",temp)
                print("target:",msg2str(targe_message))
                print("hamming distance with temp with target_message:",hamming_distance(temp,msg2str(targe_message)))  

